In [1]:
import tensorflow as tf
import malaya_speech
import malaya_speech.train
import malaya_speech.train.model.vggvox_v1 as vggvox_v1
import malaya_speech.train.model.vggvox_v2 as vggvox_v2
from malaya_speech.train.model import melgan
import numpy as np

In [2]:
resnet = vggvox_v1.model.Resnet1D(
    tf.placeholder(tf.float32, (None, 512, None, 1)), num_class = 7, mode = 'train'
)
resnet.logits

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


<tf.Tensor 'dense/BiasAdd:0' shape=(?, 7) dtype=float32>

In [3]:
resnet = vggvox_v2.model.Resnet1D(
    tf.placeholder(tf.float32, (None, 257, None, 1)), num_class = 7, mode = 'train'
)
resnet.logits

<tf.Tensor 'prediction/Softmax:0' shape=(?, 7) dtype=float32>

In [4]:
# !wget https://raw.githubusercontent.com/TensorSpeech/TensorFlowTTS/master/examples/melgan/conf/melgan.v1.yaml

In [5]:
import yaml

with open('melgan.v1.yaml', 'r') as stream:
    config = yaml.safe_load(stream)

In [6]:
generator = melgan.model.TFMelGANGenerator(
    melgan.config.MelGANGeneratorConfig(**config['melgan_generator_params']),
    name='melgan-generator',
)
discriminator = melgan.model.TFMelGANMultiScaleDiscriminator(
    melgan.config.MelGANDiscriminatorConfig(**config['melgan_discriminator_params']),
    name='melgan-discriminator',
)

In [7]:
mels_loss = melgan.loss.TFMelSpectrogram()

In [12]:
y = tf.placeholder(tf.float32, (None, None))
x = tf.placeholder(tf.float32, (None, None, 80))
y_hat = generator(x)
d_hat = discriminator(y_hat)
d = discriminator(tf.expand_dims(y, -1))

In [15]:
spect_loss = malaya_speech.train.loss.calculate_2d_loss(
    y, tf.squeeze(y_hat, -1), loss_fn = mels_loss
)

In [16]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [17]:
y_hat_, loss, d_, d_hat_ = sess.run([y_hat, spect_loss, d_hat, d], 
                                    feed_dict = {x: np.random.uniform(size=(1,200,80)),
                                                 y: np.random.uniform(size=(1,51200))})

UnimplementedError: The Conv2D op currently does not support grouped convolutions on the CPU. A grouped convolution was attempted to be run because the input depth of 16 does not match the filter input depth of 4
	 [[node melgan-discriminator_2/melgan_discriminator_scale_._2/group_conv1d_8/conv1d (defined at /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'melgan-discriminator_2/melgan_discriminator_scale_._2/group_conv1d_8/conv1d':
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frame

In [15]:
y_hat_.shape, loss

((1, 51200, 1), array([2.3302898], dtype=float32))